<a href="https://colab.research.google.com/github/dorienh/jesse/blob/Secondary/skyital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## prep and module loading

In [ ]:
!pip install tsfresh
!pip install scipy>=1.5 # need to restart runtime after installing this

     |████████████████████████████████| 102kB 6.3MB/s 
     |████████████████████████████████| 706kB 14.9MB/s 
     |████████████████████████████████| 1.1MB 56.6MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
     |████████████████████████████████| 112kB 68.1MB/s 
     |████████████████████████████████| 1.3MB 56.0MB/s 
ERROR: googleapis-common-protos 1.53.0 has requirement protobuf>=3.12.0, but you'll have protobuf 3.11.2 which is incompatible.
ERROR: google-api-core 1.26.3 has requirement protobuf>=3.12.0, but you'll have protobuf 3.11.2 which is incompatible.
ERROR: distributed 2021.5.1 has requirement dask==2021.05.1, but you'll have dask 2.12.0 which is incompatible.
ERROR: stumpy 1.8.0 has requirement scipy>=1.5, but you'll have scipy 1.4.1 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: distributed 1.25.3
    Uninstalling distri

Restart runtime after running the above. 


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/herremans_data/src/

Mounted at /content/drive
/content/drive/My Drive/herremans_data/src


In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from datapreprocess import Preprocess
from wavenetmodel import WaveNet_n_to_1
from loss import FocalLoss
from utils import custom_print
import torch

## Preprocess and save


In [ ]:
y_cols = [
    "Top_p15_a4",
    "Btm_p15_a4",
    "Buy_p15_a4",
    "Sell_p15_a4",
    "Top_p40_a1",
    "Btm_p40_a1",
    "Buy_p40_a1",
    "Sell_p40_a1",
    "ODR",
    "Top",
    "Btm",
    "Trend",
    "WM",
    "last_pivot",
]

cols_to_pred = ["Buy_p40_a1"]

datadir = "../dataset/daily/"

process = Preprocess(datadir, y_cols, cols_to_pred, last_x_days=40)

# NOTE Uncomment BELOW and see how this work
# X_train,y_train,X_test,y_test,X_val,y_val = process.process()
# print(f"X_train Shape {X_train.shape}")
# print(f"X_test Shape {X_test.shape}")
# print(f"X_val Shape {X_val.shape}")
# print(f"y_train Shape {y_train.shape}")
# print(f"y_val Shape {y_val.shape}")
# print(f"y_test Shape {y_test.shape}")
# X_train,y_train,X_test,y_test,X_val,y_val = process.prepare_tensors()
# print(f"X_train Shape {X_train.shape}")
# print(f"X_test Shape {X_test.shape}")
# print(f"X_val Shape {X_val.shape}")
# print(f"y_train Shape {y_train.shape}")
# print(f"y_val Shape {y_val.shape}")
# print(f"y_test Shape {y_test.shape}")

dataloaders = process.prepare_data_loaders(batch_size=600)



Filling Nan Values ✓
Imputing Nan Values ✓
Split Dataset into Train,Val,Test Set ✓
Min Max Normalisation on Dataset ✓
X_train Shape (3010, 40, 192), Y_train Shape (3010, 1)
X_test Shape (862, 40, 192), Y_test Shape(862, 1)
torch.Size([3010, 2]) torch.Size([431, 2])


## Saving let's colab run out of memory, only run if you have a pro account. 

In [ ]:
import pickle

with open('dataloaders.pickle', 'wb') as handle:
    pickle.dump(dataloaders, handle, protocol=pickle.HIGHEST_PROTOCOL)
    handle.close()


In [ ]:
with open('dataloaders.pickle', 'rb') as handle:
    dataloaders = pickle.load(handle)

EOFError: ignored

## Define model

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from wavenetmodel import WaveNet_n_to_1
from lstmmodel import LstmModel_n_to_1
# from loss import FocalLoss

num_epochs = 20  # 100 epochs
learning_rate = 0.003  # 0.001 lr

hidden_size = 8  # number of features in hidden state
num_layers = 3  # number of stacked lstm layers
num_classes = 2  # number of output classes

device = torch.device("cuda:0")  #changed from: cpu


model_type = "Wavenet" # "Lstm" or "Wavenet"

if model_type == "Wavenet":
    model = WaveNet_n_to_1(
        num_classes=num_classes,
        input_size=process.inputshape[2],
        seq_length=process.inputshape[1],
        device=device,
    )
    checkpointdir = "./wavenet_model.pth"
elif model_type == 'Lstm':
    model = LstmModel_n_to_1(
        num_classes=num_classes,
        input_size=process.inputshape[2],
        hidden_size=hidden_size,
        num_layers=num_layers,
        seq_length=process.inputshape[1],
        device=device,
    )
    checkpointdir = "./lstm_model.pth"

# criterion = FocalLoss(gamma=2, alpha=0.25)    #this causes and error, so trying with BCELoss() but that gives an error too about dimensions. 
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.model.parameters(), lr=learning_rate)



In [ ]:
custom_print(f"Training With {process.inputshape[2]} Features", header=True)
model.train(
    epochs=num_epochs,
    dataloader=dataloaders,
    optimizer=optimizer,
    criterion=criterion,
    checkpointdir=checkpointdir,
)

  0%|          | 0/6 [00:00<?, ?it/s]

Training With 192 Features


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch: 1 	Training Loss: 0.029890 	Validation Loss: 0.081300
validation loss decreased from inf to 0.081300. Model is being saved to ./wavenet_model.pth. True


  0%|          | 0/6 [00:00<?, ?it/s]

...

Epoch: 18 	Training Loss: 0.021914 	Validation Loss: 0.072374
validation loss decreased from 0.078342 to 0.072374. Model is being saved to ./wavenet_model.pth. True


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch: 19 	Training Loss: 0.021012 	Validation Loss: 0.077596


100%|██████████| 1/1 [00:00<00:00, 16.47it/s]

Epoch: 20 	Training Loss: 0.019295 	Validation Loss: 0.098171


Load and test

In [ ]:
model.load(checkpointdir=checkpointdir)

model.predict(testdata=process.X_test, target=process.y_test)


TypeError: ignored